# Algoritmo de classificação semi-supervisionado para classificar as notícias entre E, S, G ou Outros

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [1]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from noticias_processamento_texto import lematizador, remove_pontuacao, limpar_texto, lista_stopwords_classificador_esg
import pickle
import warnings
warnings.filterwarnings('ignore')
from classificador_esg import treina_classificador_esg, aplica_classificador_esg

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Leitura da base sintetica (chat gpt e treino semi-supervisionado)

In [2]:
# aplica o treinamento com 3 modelos diferentes
df = pd.read_excel('datasets/gpt_noticias_ESG.xlsx')
df['Texto'] = df['Texto']
df['Texto'] = df['Texto'].str.replace('Empresa [A-Z]{1,3} ', 'Empresa ', regex=True)
df['Texto'] = df['Texto'].str.replace('Texto [0-9]{1,2}: ', '', regex=True)
df['Texto'] = df['Texto'].str.replace(r'^[0-9]\. ', '', regex=True)

df_nao_rotulado = pd.read_excel('datasets/base_noticias.xlsx')

modelos = [ 'SVC', 'LogisticRegression' ,'MultinomialNB']

for m in modelos:

    print(m + ':')
    vect, model = treina_classificador_esg(df, df_nao_rotulado, col_texto='Texto', 
                                   col_classe='Dimensão', modelo=m,
                                  semi_supervisionado=True)

    pickle.dump(vect, open('models/' + str(model).replace('(', '').replace(')', '').replace("'", "") + 
                           '_vectorizer_esg_classifier.sav', 'wb'))
    pickle.dump(model, open('models/' + str(model).replace('(', '').replace(')', '').replace("'", "") + 
                            '_classifier_esg_classifier.sav', 'wb'))




SVC:
### Iteração 1 ####################################
Tamanho base total rotulada: 682
Performance na base de treinamento
accuracy:  0.9736070381231672
precision:  0.9693456915678059
recall:  0.9721908575508278
f1:  0.9706765646271018
Incluindo 7025 observações automáticas.
Performance na base de teste - 228 casos
accuracy:  0.9473684210526315
precision:  0.9439084655037644
recall:  0.9438367241032375
f1:  0.9430269461446326
### Iteração 2 ####################################
Tamanho base total rotulada: 5134
Performance na base de treinamento
accuracy:  0.8332684067004286
precision:  0.8599478450344216
recall:  0.8651858610717711
f1:  0.86236142305812
Incluindo 6391 observações automáticas.
Performance na base de teste - 228 casos
accuracy:  0.9342105263157895
precision:  0.9298268924110917
recall:  0.9285175751670673
f1:  0.9283699984058664
### Iteração 3 ####################################
Tamanho base total rotulada: 6355
Performance na base de treinamento
accuracy:  0.83335955

Incluindo 0 observações automáticas.
Performance na base de teste - 228 casos
accuracy:  0.7368421052631579
precision:  0.8837356979405033
recall:  0.6646441793500617
f1:  0.7196044987594941
MultinomialNB:
### Iteração 1 ####################################
Tamanho base total rotulada: 682
Performance na base de treinamento
accuracy:  0.9721407624633431
precision:  0.9703021780199598
recall:  0.9673460211422699
f1:  0.9685571414331393
Incluindo 4201 observações automáticas.
Performance na base de teste - 228 casos
accuracy:  0.9517543859649122
precision:  0.9530187944713451
recall:  0.9433424532603739
f1:  0.9466487027243314
### Iteração 2 ####################################
Tamanho base total rotulada: 3372
Performance na base de treinamento
accuracy:  0.7927046263345195
precision:  0.8312520050196479
recall:  0.7915277975597018
f1:  0.8084471445749734
Incluindo 5234 observações automáticas.
Performance na base de teste - 228 casos
accuracy:  0.7982456140350878
precision:  0.79846938

## Testando o modelo - SVC

In [3]:
# teste do modelo
vect = pickle.load(open('models/SVCC=0.2, kernel=linear, probability=True_vectorizer_esg_classifier.sav', 'rb'))
model = pickle.load(open('models/SVCC=0.2, kernel=linear, probability=True_classifier_esg_classifier.sav', 'rb'))


df_teste = pd.read_excel('datasets/gpt_noticias_ESG.xlsx')
df_teste = df_teste[~pd.isnull(df_teste['Texto'])]
aplica_classificador_esg(vect, model, df_teste, 
                             comparar_com_real=True, col_texto_origem='Texto', 
                             col_texto_saida='texto_ajustado', col_classe_verdadeira='Dimensão')

Performance na base de teste - 910 casos
accuracy:  0.9307692307692308
precision:  0.9261636152452077
recall:  0.9275541795665635
f1:  0.9265680460247835


array(['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'E',
       'S', 'S', 'S', 'S', 'S', 'S', 'S', 'G', 'G', 'G', 'G', 'G', 'G',
       'G', 'G', 'G', 'G', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E',
       'E', 'S', 'S', 'E', 'S', 'Outros', 'G', 'S', 'G', 'G', 'S', 'G',
       'G', 'S', 'E', 'G', 'S', 'G', 'G', 'S', 'G', 'E', 'E', 'E', 'E',
       'E', 'E', 'E', 'E', 'E', 'E', 'S', 'Outros', 'S', 'S', 'S', 'S',
       'S', 'S', 'S', 'S', 'G', 'G', 'G', 'S', 'G', 'G', 'G', 'G', 'S',
       'G', 'Outros', 'Outros', 'Outros', 'Outros', 'Outros', 'Outros',
       'Outros', 'Outros', 'E', 'Outros', 'Outros', 'Outros', 'Outros',
       'Outros', 'Outros', 'E', 'Outros', 'Outros', 'Outros', 'Outros',
       'Outros', 'Outros', 'Outros', 'Outros', 'Outros', 'Outros',
       'Outros', 'Outros', 'E', 'Outros', 'E', 'E', 'E', 'E', 'E', 'G',
       'E', 'E', 'E', 'E', 'G', 'G', 'G', 'G', 'G', 'S', 'G', 'S', 'G',
       'G', 'S', 'S', 'E', 'E', 'S', 'S', 'S', 'E', 'S', 'S', 'Outros